<a href="https://colab.research.google.com/github/stepkurniawan/network-analysis-wikipedia-journals/blob/main/2_Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import itertools

In [6]:
df_journal.columns

NameError: ignored

In [ ]:
g = nx.Graph()

In [ ]:
def draw_network(graph, seed=42):
    nx.draw(graph, pos=nx.spring_layout(graph, seed=seed),
            with_labels=True, node_size=1000)

In [ ]:
g.add_edges_from([['Stephen', 'Jenya'], ['Stephen', 'Beka'], ['Beka', 'Jenya']])
g.add_edges_from([['Stephen', 'Jenya'], ['Stephen', 'Jenya']])
g.add_nodes_from(['A'])
#, 'Jenya'], ['Stephen', 'Beka'], ['Beka', 'Jenya']])
draw_network(g)

In [ ]:
df_journal = pd.read_csv('df_journal.csv')
df_journal

In [ ]:
df_journal.nunique()

In [ ]:
#G = nx.from_pandas_edgelist(df_journal, 'page_title', 'journal_name')

In [ ]:
# from matplotlib.pyplot import figure
# figure(figsize=(10, 8))
# nx.draw_shell(G, with_labels=True)

In [ ]:
df_sample_journal_data = pd.DataFrame(np.array([['p1', 'j1'],['p1', 'j2'], ['p1', 'j3'], ['p2', 'j1'], ['p2', 'j2']]),
                   columns=['page_title', 'journal_name'])
df_sample_journal_data

In [ ]:
# df_groupby = df_sample_journal_data.groupby('page_title')['journal_name'].apply(lambda x: ', '.join(x))
df_groupby = df_sample_journal_data.groupby('page_title')['journal_name'].apply(list)

df_groupby

In [ ]:
iterations_sample = itertools.combinations(df_groupby['p1'], 2)
print(iterations_sample)
for i in iterations_sample:
    print(i)

In [ ]:
df_groupby = df_journal.groupby('page_title')['journal_name'].apply(list)
df_groupby

In [ ]:
# find combination of all the journal name

iterations = itertools.combinations(df_groupby[2], 2)

for combi in iterations: 
    print(combi)

In [ ]:
# make list iterations based on the combinations

journal_relation_list = []

for page in range(len(df_groupby)):
    iterations = itertools.combinations(df_groupby[page], 2)
    for iter in iterations:
        journal_relation_list.append(iter)

#journal_relation_list


In [ ]:
df1 = pd.DataFrame (journal_relation_list, columns = ['journal1', 'journal2'])

df1

In [ ]:
df1['journal1'] = df1['journal1'].astype(str)
df1['journal2'] = df1['journal2'].astype(str)
df1

In [ ]:
df2 = df1[['journal1','journal2']] = pd.DataFrame(np.sort(df1[['journal1','journal2']].values), 
                                 index=df1.index, columns=['journal1','journal2'])
df2

In [ ]:
df3 = df2[['journal1', 'journal2']].value_counts().reset_index(name='Weight')
df3

In [ ]:
df3 = df3[df3.Weight>=200]
df3

In [ ]:
df3.hist(column='Weight')